# Watson Speech to Textのカスタマイズ練習用Notebook

- [Watson Python SDK](https://github.com/watson-developer-cloud/python-sdk/blob/master/ibm_watson/speech_to_text_v1.py)
- データ(MP3や用語辞書、トランスクリプト)も含んでいるので、資格情報だけ代えればこのままでも動きます
- ご自身のデータで実行するなら、データを準備の上、1.準備の 固定値のセルの値を変更してください
- 2019/09/11 時点でSDK V4(ibm-watson)で動くように変更しました

# 1.準備

## WatsonのPython SDKを未導入なら導入 [説明](https://github.com/watson-developer-cloud/python-sdk)

In [48]:
#!pip install --upgrade ibm-watson
from ibm_watson import SpeechToTextV1
import json

## IBM CloudのSTTサービスの資格情報(Credential)を丸ごとコピペ ( 使うのはapikeyだけ）
## ★STTのカスタマイズにはPAYGインスタンスが必要です。Liteプランではできません。

In [2]:
Credentials ={
  "apikey": "************************************LavjavD",
  "iam_apikey_description": "Auto-generated for key ********-****-4ca2-*************c720",
  "iam_apikey_name": "Auto-generated service credentials",
  "iam_role_crn": "crn:v1:bluemix:public:iam::::serviceRole:Manager",
  "iam_serviceid_crn": "crn:v1:bluemix:public:iam-identity::a/********************************::serviceid:ServiceId-********-6220-**********************",
  "url": "https://stream.watsonplatform.net/speech-to-text/api"
}

## インスタンス作成

In [3]:
stt = SpeechToTextV1(
    iam_apikey=Credentials["apikey"],
    url=Credentials["url"])

## 固定値のセット
### YouTubeの動画 [IBM Q - 量子コンピューター最新情報ならここ！](https://www.youtube.com/watch?v=-CJNJ1ETV5Q&t=33s)から抜いた音声(mp3)を利用

In [4]:
AUDIO_FILE = "mp3/IBMQ-01.mp3"
AUDIO_FORMAT = "audio/mp3"
BASE_MODEL = "ja-JP_BroadbandModel"
TRANSCRIPT_TEXT="Transcript.txt"   # コーパス
TRANSCRIPT_JSON="Transcript.json"  # 用語辞書
ACOUSTIC_FILE = "mp3/IBMQ-10min.mp3"

# 2.通常の（カスタマイズなし）認識

In [5]:
def recognizeNormal(audio_file):
    with open(audio_file,'rb') as audio_file:
        return  stt.recognize(
                      audio=audio_file,
                      content_type=AUDIO_FORMAT,
                      model=BASE_MODEL,
                      language_customization_id=None,
                      acoustic_customization_id=None,
                      base_model_version=None,
                      customization_weight=None,
                      inactivity_timeout=None,
                      keywords=None,
                      keywords_threshold=None,
                      max_alternatives=None,
                      word_alternatives_threshold=None,
                      word_confidence=None,
                      timestamps=None,
                      profanity_filter=None,
                      smart_formatting=None,
                      speaker_labels=None,
                      customization_id=None,
                      grammar_name=None,
                      redaction=None)
    

In [7]:
recognizeNormal(AUDIO_FILE).get_result()

{'result_index': 0,
 'results': [{'alternatives': [{'confidence': 0.65,
     'transcript': 'ええ 、 役立た ＦＬＯＰＳ なんです けれども 並列 度 って いう 観点 から 見れば ええ 一台 の 報道 を 見て みると D_エー 買わない の プロセッサー が 二つ D_エー それでは 二十 二個 D_アノー 四 ＳＭＰ な やつ ＋ 一台 に ええ ＣＴＵ エミリア 文字 部屋 が 六枚 '}],
   'final': True},
  {'alternatives': [{'confidence': 0.78,
     'transcript': 'あさって います で それぞれ D_エー 五千 百 二十個 と いう D_エー 凄い 並列 なんです けれども ですから 一 ノード で 三万 を 超える 並列 度 が あります '}],
   'final': True}]}

# 3.カスタム言語モデル

## 現状のカスタム言語モデルをリストする

In [9]:
# 整理のため、いったん全部削除する
stt.delete_language_model(customization_id='85f64389-2fa3-4ce0-9e98-d8740b9dd1cc').get_result()
stt.delete_language_model(customization_id='b2fa6e33-af67-4698-bb14-d04a2c091aaf').get_result()

{}

In [10]:
stt.list_language_models(language=None).get_result()

{'customizations': []}

## カスタム言語モデルを作成する => IDを入手

In [11]:
result = stt.create_language_model(
      name="IBMQ_LanguageModel",
      base_model_name=BASE_MODEL,
      dialect=None,
      description="IBMQ_LanguageModel")

In [12]:
language_customization_id = result.get_result()["customization_id"]
print("language_customization_id:", language_customization_id)

language_customization_id: 531b4187-3029-4758-8adc-66b531a77096


## カスタム言語モデルへのコーパスの追加

In [13]:
# v4での変更点
with open(TRANSCRIPT_TEXT,'r',encoding="utf-8") as f:
    stt.add_corpus(
       customization_id=language_customization_id,
       corpus_name="IBMQ_Transcrpt",
       corpus_file= f,
       allow_overwrite=True).get_result()

## カスタム言語モデルへの用語の追加（非同期処理)

In [14]:
with open(TRANSCRIPT_JSON,'r',encoding="utf-8") as f:
    custom_words = json.load(f)["words"]
    stt.add_words(
         customization_id=language_customization_id,
         words=custom_words).get_result()   

## 用語追加の完了状況をチェック( 'status': 'ready'になるまで）

In [16]:
stt.get_language_model(customization_id=language_customization_id).get_result()

{'base_model_name': 'ja-JP_BroadbandModel',
 'created': '2019-09-11T04:09:03.856Z',
 'customization_id': '531b4187-3029-4758-8adc-66b531a77096',
 'description': 'IBMQ_LanguageModel',
 'dialect': 'ja-JP',
 'language': 'ja-JP',
 'name': 'IBMQ_LanguageModel',
 'owner': '810043a9-677c-4dc7-b35e-e0524f92e2eb',
 'progress': 0,
 'status': 'ready',
 'updated': '2019-09-11T04:10:47.193Z',
 'versions': ['ja-JP_BroadbandModel.v2019-04-26']}

## 登録されている用語の確認 

In [17]:
stt.list_words(customization_id=language_customization_id, word_type=None, sort=None).get_result()

{'words': [{'count': 1,
   'display_as': '200',
   'sounds_like': ['ニヒャク'],
   'source': ['user'],
   'word': '200'},
  {'count': 1,
   'display_as': 'GPU',
   'sounds_like': ['ジーピーユー'],
   'source': ['user'],
   'word': 'GPU'},
  {'count': 1,
   'display_as': 'NVIDIA',
   'sounds_like': ['エヌビディア'],
   'source': ['user'],
   'word': 'NVIDIA'},
  {'count': 1,
   'display_as': 'Power9',
   'sounds_like': ['パワーナイン'],
   'source': ['user'],
   'word': 'Power9'},
  {'count': 1,
   'display_as': 'SMT',
   'sounds_like': ['エスエムティ'],
   'source': ['user'],
   'word': 'SMT'},
  {'count': 1,
   'display_as': 'コア',
   'sounds_like': ['コア'],
   'source': ['user'],
   'word': 'コア'},
  {'count': 3,
   'display_as': 'コヒー',
   'sounds_like': ['コヒー'],
   'source': ['IBMQ_Transcrpt'],
   'word': 'コヒー'},
  {'count': 1,
   'display_as': 'ノード',
   'sounds_like': ['ノード'],
   'source': ['user'],
   'word': 'ノード'},
  {'count': 2,
   'display_as': 'ペタ',
   'sounds_like': ['ペタ'],
   'source': ['IBMQ_Transcrpt']

In [18]:
# エラーがあれば個別に消す
#stt.delete_word(customization_id=language_customization_id, word_name='xxx').get_result()

## カスタム言語モデルのトレーニング実施

In [19]:
stt.train_language_model(
    customization_id=language_customization_id,
    word_type_to_add=None,
    customization_weight=None).get_result()

{}

## トレーニングステータスの確認（status=availableになるまで）

In [23]:
stt.get_language_model(language_customization_id).get_result()

{'base_model_name': 'ja-JP_BroadbandModel',
 'created': '2019-09-11T04:09:03.856Z',
 'customization_id': '531b4187-3029-4758-8adc-66b531a77096',
 'description': 'IBMQ_LanguageModel',
 'dialect': 'ja-JP',
 'language': 'ja-JP',
 'name': 'IBMQ_LanguageModel',
 'owner': '810043a9-677c-4dc7-b35e-e0524f92e2eb',
 'progress': 100,
 'status': 'available',
 'updated': '2019-09-11T04:11:47.213Z',
 'versions': ['ja-JP_BroadbandModel.v2019-04-26']}

## カスタム言語モデルを使った音声to テキスト変換(STT)実行

In [24]:
def recognizeWithLangModel(audio_file):
    with open(audio_file,'rb') as audio_file:
        return stt.recognize(
                      audio=audio_file,
                      content_type= AUDIO_FORMAT,
                      model= BASE_MODEL,
                      language_customization_id=language_customization_id,
                      acoustic_customization_id=None,
                      base_model_version=None,
                      customization_weight=None,
                      inactivity_timeout=None,
                      keywords=None,
                      keywords_threshold=None,
                      max_alternatives=None,
                      word_alternatives_threshold=None,
                      word_confidence=None,
                      timestamps=None,
                      profanity_filter=None,
                      smart_formatting=None,
                      speaker_labels=None,
                      customization_id=None,
                      grammar_name=None,
                      redaction=None)

In [25]:
print(recognizeWithLangModel(AUDIO_FILE).result)

{'result_index': 0, 'results': [{'alternatives': [{'confidence': 0.9, 'transcript': 'ええ で 二百 ペタフロップス なんです けれども 並列度 どう って いう 観点 から 見れば ええ 一台 の ノード を 見て みると D_エー Power9 の プロセッサー が 二つ D_エー それぞれ 二十 二 コア の 四 Ｓ ＭＴ の やつ プラス 一台 に ええ ＧＰ Ｕ NVIDIA の ＧＰ Ｕ が 六枚 刺さって ます で それぞれ D_エー 五千 百 二十 コア と いう D_エー 凄い 並列度 の です けれども ですから 一 ノード で 三万 を 超える 並列度 道 が あります '}], 'final': True}]}


# 4.カスタム音響モデル

## 音響モデルの作成

In [44]:
# 整理のため不要なものを消す
#stt.delete_acoustic_model(customization_id="213ca981-a3d7-4e59-824a-1dbabb4638ef").get_result()
#stt.delete_acoustic_model(customization_id="dd4315eb-51c0-44be-94bd-ea7bf4b47639").get_result()

In [26]:
stt.list_acoustic_models(language=None).get_result()

{'customizations': []}

In [27]:
result = stt.create_acoustic_model(
    name="IBMQ_AcousticModel",
    base_model_name= BASE_MODEL,
    description="IBMQ_AcousticModel")
result.get_result()

{'customization_id': '13ac8027-84a8-4be5-b9a7-9e7ff7de5efd'}

In [28]:
acoustic_customization_id = result.get_result()["customization_id"]
print(acoustic_customization_id)

13ac8027-84a8-4be5-b9a7-9e7ff7de5efd


## 定義情報の確認

In [29]:
stt.get_acoustic_model(customization_id=acoustic_customization_id).get_result()

{'base_model_name': 'ja-JP_BroadbandModel',
 'created': '2019-09-11T04:13:05.890Z',
 'customization_id': '13ac8027-84a8-4be5-b9a7-9e7ff7de5efd',
 'description': 'IBMQ_AcousticModel',
 'language': 'ja-JP',
 'name': 'IBMQ_AcousticModel',
 'owner': '810043a9-677c-4dc7-b35e-e0524f92e2eb',
 'progress': 0,
 'status': 'pending',
 'updated': '2019-09-11T04:13:05.890Z',
 'versions': ['ja-JP_BroadbandModel.v2019-04-26']}

## 音声サンプルを登録(最低10分-MAX 100MB)　非同期

In [30]:
with open(ACOUSTIC_FILE,'rb') as sample_audio:
    stt.add_audio(
                  customization_id=acoustic_customization_id,
                  audio_name="IBM-Q-10min",
                  audio_resource=sample_audio,
                  content_type= AUDIO_FORMAT,
                  contained_content_type=None,
                  allow_overwrite=True).get_result()

## 登録の処理状況をチェック('status': 'ok'?)

In [34]:
stt.get_audio(customization_id=acoustic_customization_id, audio_name="IBM-Q-10min").get_result()

{'details': {'codec': 'mp3', 'frequency': 44100, 'type': 'audio'},
 'duration': 654,
 'name': 'IBM-Q-10min',
 'status': 'ok'}

## 音響モデルのトレーニングの実行(非同期実行）- カスタム言語モデルも併用

In [35]:
stt.train_acoustic_model(
    customization_id=acoustic_customization_id,
    custom_language_model_id=language_customization_id).get_result()

{}

##  トレーニングのステータスの確認 (status = available?)

In [45]:
stt.get_acoustic_model(customization_id=acoustic_customization_id).get_result() # check status

{'base_model_name': 'ja-JP_BroadbandModel',
 'created': '2019-09-11T04:13:05.890Z',
 'customization_id': '13ac8027-84a8-4be5-b9a7-9e7ff7de5efd',
 'description': 'IBMQ_AcousticModel',
 'language': 'ja-JP',
 'name': 'IBMQ_AcousticModel',
 'owner': '810043a9-677c-4dc7-b35e-e0524f92e2eb',
 'progress': 100,
 'status': 'available',
 'updated': '2019-09-11T04:23:46.875Z',
 'versions': ['ja-JP_BroadbandModel.v2019-04-26']}

## 音響モデルで認識を実行する

In [46]:
def reconnizeWithAcousticModel(audio_file):
    with open(audio_file,'rb') as audio_file:
        return  stt.recognize(
                      audio=audio_file,
                      content_type=AUDIO_FORMAT,
                      model=BASE_MODEL,
                      language_customization_id=language_customization_id,
                      acoustic_customization_id=acoustic_customization_id,
                      base_model_version=None,
                      customization_weight=None,
                      inactivity_timeout=None,
                      keywords=None,
                      keywords_threshold=None,
                      max_alternatives=None,
                      word_alternatives_threshold=None,
                      word_confidence=None,
                      timestamps=None,
                      profanity_filter=None,
                      smart_formatting=None,
                      speaker_labels=None,
                      customization_id=None,
                      grammar_name=None,
                      redaction=None)


In [47]:
print(reconnizeWithAcousticModel(AUDIO_FILE).result)

{'result_index': 0, 'results': [{'alternatives': [{'confidence': 0.96, 'transcript': 'ええ で 二百 ペタフロップス なんです けれども 並列度 どう って いう 観点 から 見れば ええ 一台 の ノード を 見て みると D_エー Power9 の プロセッサー が 二つ D_エー それぞれ 二十 二 コア の 四 Ｓ ＭＴ の やつ プラス 一台 に ＧＰ Ｕ NVIDIA の ＧＰ Ｕ が 六枚 '}], 'final': True}, {'alternatives': [{'confidence': 0.91, 'transcript': '刺さって ます で それぞれ D_エー 五千 百 二十 コア と いう D_エー 凄い 並列度 の です けれども ですから 一 ノード で 三万 を 超える 並列度 道 が あります '}], 'final': True}]}
